In [17]:
using SQLite, BioStructures, DataFrames

In [18]:
db_instance = SQLite.DB("./HashedData/quickret.db");

In [8]:
function write_proteinstructure(db_instance::SQLite.DB, structure_name::String)
    DBInterface.execute(db_instance, "INSERT INTO proteinstructure (name) VALUES ('$(structure_name)')")
end

function write_proteinstructure(db_instance::SQLite.DB, protein_structure::ProteinStructure)
    DBInterface.execute(db_instance, "INSERT INTO proteinstructure (name) VALUES ('$(protein_structure.name)')")    
end

write_proteinstructure (generic function with 2 methods)

In [9]:
function write_model(db_instance::SQLite.DB, model_number::Int, protein_structure_id::Int)
   DBInterface.execute(db_instance, "INSERT INTO model (number,proteinstructureid) VALUES ($(model_number),$(protein_structure_id))") 
end

function write_model(db_instance::SQLite.DB, model::Model, protein_structure_id::Int)
   DBInterface.execute(db_instance, "INSERT INTO model (number,proteinstructureid) VALUES ($(model.number),$(protein_structure_id))")
end

write_model (generic function with 2 methods)

In [10]:
function write_residue(db_instance::SQLite.DB, residue::Residue, chain_id::Int)
    DBInterface.execute(db_instance, "INSERT INTO residue (name, number, ins_code, het_res, chainid) 
        VALUES ('$(residue.name)', $(residue.number), '$(residue.ins_code)', $(residue.het_res), $(chain_id))")
end

write_residue (generic function with 1 method)

In [11]:
function write_atom(db_instance::SQLite.DB, atom::Atom, residue_id::Int)
    DBInterface.execute(db_instance, "INSERT INTO atom 
        (serial, name, alt_loc_id, xcoord, ycoord, zcoord, occupancy, element, charge, residueid) 
        VALUES ($(atom.serial), '$(atom.name)', '$(atom.alt_loc_id)', $(x(atom)), $(y(atom)), $(y(atom)), $(atom.occupancy), '$(atom.element)', '$(atom.charge)', $(residue_id))")
end

write_atom (generic function with 1 method)

In [12]:
function get_proteinstructure_id(db_instance::SQLite.DB, struc::ProteinStructure)
    
    return (DBInterface.execute(db_instance, "
        SELECT proteinstructureid FROM proteinstructure WHERE name = '$(struc.name)'
        ") |> DataFrame).proteinstructureid[1]
    
end
function get_proteinstructure_id(db_instance::SQLite.DB, model::Model)
   return (DBInterface.execute(db_instance, "
        SELECT proteinstructureid FROM proteinstructure WHERE name = '$(model.structure.name)'        
        ") |> DataFrame).proteinstructureid[1]
end
function get_proteinstructure_id(db_instance::SQLite.DB, chain::Chain)
    return (DBInterface.execute(db_instance, "
                SELECT proteinstructureid FROM proteinstructure WHERE name = '$(chain.model.structure.name)'        
 ") |> DataFrame)proteinstructureid[1]
end

get_proteinstructure_id (generic function with 3 methods)

In [13]:
function get_model_id(db_instance::SQLite.DB, model::Model)
    return (DBInterface.execute(db_instance, "
            SELECT modelid FROM model WHERE number = $(model.number)
            ") |> DataFrame).modelid[1]
end
# function get_model_id(db_instance::SQLite.DB, chain::Chain)
#     return (DBInterface.execute(db_instance, "
#             SELECT modelid FROM model WHERE number = $(chain.model.number)
#             ") |> DataFrame).modelid
# end
function get_model_id(db_instance::SQLite.DB, chain::Chain)
   return (DBInterface.execute(db_instance, "
        SELECT 
            model.modelid
        FROM 
            model
            INNER JOIN proteinstructure ON proteinstructure.proteinstructureid = model.proteinstructureid
        WHERE
            proteinstructure.name = '$(chain.model.structure.name)'
            AND
            model.number = $(chain.model.number)
        
    ") |> DataFrame).modelid[1]
end

get_model_id (generic function with 2 methods)

In [14]:
function get_chain_id(db_instance::SQLite.DB, chain::Chain)
    return (DBInterface.execute(db_instance, "
            SELECT chainid FROM chain WHERE id = $(chain.id)
            ") |> DataFrame).chainid[1]
end
function get_chain_id(db_instance::SQLite.DB, residue::Residue)
    return (DBInterface.execute(db_instance, "
        SELECT 
            chain.chainid
        FROM 
            chain
            INNER JOIN model ON model.modelid = chain.modelid
            INNER JOIN proteinstructure ON proteinstructure.proteinstructureid = model.proteinstructureid
        WHERE
            proteinstructure.name = '$(residue.chain.model.structure.name)'
            AND
            model.number = $(residue.chain.model.number)
            AND
            chain.id = '$(residue.chain.id)'
    ") |> DataFrame).chainid[1]
    
end

get_chain_id (generic function with 2 methods)

In [15]:
function get_residue_id(db_instance::SQLite.DB, residue::Residue)
    return (DBInterface.execute(db_instance, "
            SELECT residueid FROM residue WHERE number = $(residue.number)
            ") |> DataFrame).residueid[1]
end
function get_residue_id(db_instance::SQLite.DB, atom::Atom)
    return (DBInterface.execute(db_instance, "
        SELECT 
            residue.residueid
        FROM 
            residue
            INNER JOIN chain ON chain.chainid = residue.chainid
            INNER JOIN model ON model.modelid = chain.modelid
            INNER JOIN proteinstructure ON proteinstructure.proteinstructureid = model.proteinstructureid
        WHERE
            proteinstructure.name = '$(atom.residue.chain.model.structure.name)'
            AND
            model.number = $(atom.residue.chain.model.number)
            AND
            chain.id = '$(atom.residue.chain.id)'
            AND
            residue.number = $(atom.residue.number)
    ") |> DataFrame).residueid[1]
    
end

get_residue_id (generic function with 2 methods)

In [16]:
function write_pdb(db_instance::SQLite.DB, struc::ProteinStructure)
    write_proteinstructure(db_instance, struc)
    for model_value in struc
       write_model(db_instance, model_value, get_proteinstructure_id(db_instance, model_value))
    end
    
    for model_value in struc
        for chain_value in model_value
            write_chain(db_instance, chain_value, get_model_id(db_instance, chain_value))
        end
    end
    
    for model_value in struc
        for chain_value in model_value
            for residue_value in chain_value
                write_residue(db_instance, residue_value, get_chain_id(db_instance, residue_value))
            end
        end
    end
end

write_pdb (generic function with 1 method)